# Mean minimal spanning tree length

Import supporting libraries

In [24]:
# imports

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from itertools import combinations_with_replacement as cwr
from functools import reduce
import sys
import json
import ipywidgets as widgets


Choose $k$, the number of particle types; i.e. the random graph will be $k$-partite.

In [25]:
k = 4;
assert k == 4, f"Interaction strength matrix is not defined for k = {k}"

### Creating helper functions

#### Factorial computation

In [26]:
def factorial(n: int):
  assert isinstance(n, int), f"Factorial function only accepts integer inputs.\n(Received {type(n)})";
  if n < 0: return np.nan;
  return 1 if ((n==1)|(n==0)) else n*factorial(n-1);

#### Define function for each summand $\{x: \langle x | 1 \rangle = n \}$

In [27]:
def summand(x: np.ndarray, V: np.ndarray):
  xm = [float(num) for num in x - 1];
  y = np.matmul(V, x)**np.array(xm)

  # compute leading constant
  c0 = factorial(int(x.sum()))

  # compute product of components of y
  c1 = reduce(
      lambda a,b: a*b,
      y,
      1
    )
  
  # compute denominator, x! = x_1! * x_2! * ... * x_k!
  c2 = reduce(
      lambda a,b: a*b,
      map(
        lambda en: factorial(int(en)), 
        x
      ),
      1
    )

  return c0 * c1 / c2;

#### Compute partitions

In [28]:
def get_partitions(n, k):
  '''
  number of ways to designate your n particles into k urns:
  place k-1 separators among/around the ordered n particles (n+1 choices)
  the particles above the highest separator are urn 1 members,
  the particles between the first & second sep are in urn 2, ...
  the placement of separators is done by choosing one of the n+1 spaces k-1 times
  with replacement and without regard to order; use the itertools function for this.
  '''
  raw_parts = list(
      cwr(np.arange(0, n+1), k-1)
    ) # weak

    # first bin:  rp[0]
    # middle bins:  np.diff(rp)
    # final bin:  n - rp[-1]

  # would be nice to perform this process using map, not listing
  out = np.empty((len(raw_parts), k));
  for ent, y in enumerate(raw_parts):
    out[ent, :] = [y[0]] + np.diff(y).tolist() + [n-y[-1]]

  return out;

#### Code for generating LHS of (23)

In [29]:
def compute_lhs(n, k, V):
  lhs = 0;
  for part in get_partitions(n,k):
    lhs += summand(part, V)
  return lhs;

#### Code for generating RHS of (23)

In [30]:
def compute_rhs(n,k):
  rhs = k * (k-1)**(n-1) * n**(n-4)
  return rhs;

Now we get into actually computing the quantities of interest. Resolution manages the resolution of the surface plot below.

In [31]:
# resolution of the 2-d plots
resolution = 50

Begin by defining the parameters $n$ (total number of particles / nodes in the graph), $p$, $q$, and $r$ (the inter-group interaction strength parameters)
We define now the domain for the independent, nonconstant variables:

In [32]:
lower_bound = 0.5;
upper_bound = 20;

domain_x = np.linspace(lower_bound, upper_bound, resolution);
domain_y = np.linspace(lower_bound, upper_bound, resolution);

[X,Y] = np.meshgrid(domain_x, domain_y)
Z = np.zeros(X.shape)

r_slider = widgets.FloatSlider(
    value=1,
    min=lower_bound,
    max=upper_bound,
    step=0.1,
    description='r:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

n_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description='n value:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


Compute the interaction strength matrix, then execute the computation steps.

#### Notice that the output matrix $Z$ is the ratio fo the left- and right-hand sides of (24):

In [36]:

def run_plot(n,var3):
    r = var3;
    for eye, p in enumerate(domain_x):
      for jay, q in enumerate(domain_y):
        V = np.array([
          [0, p, q, r],
          [p, 0, r, q],
          [q, r, 0, p],
          [r, q, p, 0]
        ])

        G = compute_lhs(n,k,V)
        F = compute_rhs(n,k)
        Z[eye, jay] = G/F;

    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)
    
    ax.set_title(f"k={k}, n={n}\nG(p, q, {r})/F")
    plt.show()

We plot here the results:

In [37]:
# Adding ipy-widgets:
%matplotlib widget

In [35]:
widgets.interact(run_plot, n=n_slider, var3=r_slider)

interactive(children=(IntSlider(value=2, continuous_update=False, description='n value:', max=20, min=1), Floa…

<function __main__.run_plot(n, var3)>